In [1]:
import h5py
file_path = '/content/drive/MyDrive/processed_data.mat'
with h5py.File(file_path, 'r') as mat_data:
    print(list(mat_data.keys()))  # List available variables
    print(mat_data['y_data'])  # Print dataset metadata


['#refs#', 'X_data', 'y_data']
<HDF5 dataset "y_data": shape (1035, 1), type "|O">


In [2]:
import h5py
import numpy as np

# Load the .mat file
file_path = '/content/drive/MyDrive/processed_data.mat'  # Adjust path if needed
with h5py.File(file_path, 'r') as mat_data:
    X_data = np.array(mat_data['X_data'])  # Load EEG data

    # Extract and decode phoneme labels correctly
    y_data = []
    y_data_ref = mat_data['y_data']  # This is an array of references

    for i in range(y_data_ref.shape[0]):  # Loop over stored labels
        ref = y_data_ref[i][0]  # Get reference to actual dataset
        label_data = mat_data[ref][:]  # Dereference and load data
        label = ''.join(map(chr, label_data.flatten()))  # Decode bytes to string
        y_data.append(label)

# Convert to NumPy array
y_data = np.array(y_data)

# Check results
print("X_data shape:", X_data.shape)
print("y_data length:", len(y_data))
print("Example phoneme labels:", y_data[:10])

X_data shape: (1035, 256, 61)
y_data length: 1035
Example phoneme labels: ['pa' 'pe' 'bi' 'bi' 'bi' 'bi' 'ta' 'de' 'di' 'pi']


In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense,
                                     Dropout, LSTM, TimeDistributed, Reshape, GlobalAveragePooling1D,
                                     Bidirectional)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import random

# Load EEG Data
file_path = '/content/drive/MyDrive/processed_data (1).mat'
with h5py.File(file_path, 'r') as mat_data:
    X_data = np.array(mat_data['X_data'])
    y_data_ref = mat_data['y_data']

    y_data = []
    for i in range(y_data_ref.shape[0]):
        ref = y_data_ref[i][0]
        label_data = mat_data[ref][:]
        label = ''.join(map(chr, label_data.flatten()))
        y_data.append(label)

# Sort EEG Data by Phoneme Label
sorted_indices = np.argsort(y_data)
X_data = X_data[sorted_indices]
y_data = np.array(y_data)[sorted_indices]

# Convert Labels to One-Hot Encoding
unique_labels = sorted(list(set(y_data)))
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
y_encoded = np.array([label_to_index[label] for label in y_data])
y_onehot = to_categorical(y_encoded, num_classes=len(unique_labels))

# Reshape EEG Data for CNN+LSTM
X_data = np.expand_dims(X_data, axis=-1)

# Data Augmentation Functions
def add_noise(data, noise_level=0.02):
    return data + np.random.normal(0, noise_level, data.shape)

def time_shift(data, shift_max=10):
    shift = random.randint(-shift_max, shift_max)
    return np.roll(data, shift, axis=1)

def scale_amplitude(data, scale_min=0.8, scale_max=1.2):
    scale = np.random.uniform(scale_min, scale_max)
    return data * scale

def frequency_mask(data, mask_size=5):
    data[:, :mask_size, :] = 0
    return data

# Applying Data Augmentation
augmented_X = []
augmented_y = []
for i in range(len(X_data)):
    augmented_X.append(X_data[i])
    augmented_X.append(add_noise(X_data[i]))
    augmented_X.append(time_shift(X_data[i]))
    augmented_X.append(scale_amplitude(X_data[i]))
    augmented_X.append(frequency_mask(X_data[i]))
    for _ in range(5):
        augmented_y.append(y_onehot[i])

X_data = np.array(augmented_X)
y_onehot = np.array(augmented_y)

# Split Data
X_train, X_val, y_train, y_val = train_test_split(X_data, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot)

# Compute Class Weights for Balanced Training
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# CNN+BiLSTM Model
def CNN_LSTM(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # CNN Feature Extractor
    x = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = TimeDistributed(Flatten())(x)
    x = Reshape((x.shape[1], x.shape[-1]))(x)

    # Bidirectional LSTM
    x = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)))(x)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.4)(x)

    # Output Layer
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

# Building Model
input_shape = (X_train.shape[1], X_train.shape[2], 1)
num_classes = len(unique_labels)
model = CNN_LSTM(input_shape, num_classes)

# Compiling Model
model.compile(optimizer=AdamW(learning_rate=0.001, weight_decay=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Training Model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=512,
                    class_weight=class_weight_dict,
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)

# Saving Model
model.save('/content/drive/MyDrive/CNN_BiLSTM_enhanced(1).h5')

# Evaluate Model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Test Accuracy: {test_acc:.2f}")

# Predicting and Printing First 5 Phonemes
predictions = model.predict(X_val)
predicted_indices = np.argmax(predictions, axis=1)
true_indices = np.argmax(y_val, axis=1)

for i in range(5):
    print(f"Predicted: {unique_labels[predicted_indices[i]]} | True: {unique_labels[true_indices[i]]}")